In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 3000)

In [3]:
df = pd.read_csv('raw_data.csv')

### Data Cleaning 

In [4]:
df.head()

,Listing ID,Title,Description,Price,Sq Ft Price,Summary
0,cardid71632357,2 BHK Apartment for Sale in Kelambakkam Chennai,"Located in Kelambakkam, OMR, Incor PBEL City is an upcoming housing project, that offers 2BHK and 3BHK flats. It is developed by Incor Infrastructure to facilitate both, undisturbed private moments and exuberant camaraderie of its residents. PBEL City is lavishly designed with modern techniques and features with the best of industry specifications.",₹ 39.9 Lac,"₹4,000 per sqft",SUPER AREA 997 sqft STATUS Ready to Move TRANSACTION New Property FURNISHING Unfurnished CAR PARKING 1 Covered BATHROOM 2
1,cardid73795355,2 BHK Builder Floor for Sale in Mudichur Chennai,"Premium Apartment located in vibrant surroundings that fit right into your budget, located next to Tambaram near to Urapakkam new bus stand on Mudichur Tambaram main road. Has supreme connectivity to GST road, Oragadam road and outer ring road. it has very close access to schools, colleges well known educational institutions. good transportation facilities near by markets and other entertainments. Apartments are designed in a very spacious, vasthu compliant with natural light and Breeze.",₹ 50.6 Lac,"₹5,800 per sqft",SUPER AREA 873 sqft UNDER CONSTRUCTION Poss. by Sep '24 TRANSACTION New Property FURNISHING Unfurnished BATHROOM 2
2,cardid72552613,1 BHK Builder Floor for Sale in Kundrathur Chennai,"Karthick Amore is the project where you can discover the epitome of city living at affordable prices. The homes in the project has spacious rooms and is constructed with 100% red bricks. Karthick Amore is located near top rated matriculation, CBSE and ICSE schools. Hospitals, Temples, Cricket grounds are all situated within 1Km Radius from the project.",₹ 27.4 Lac,"₹5,300 per sqft",SUPER AREA 517 sqft STATUS Ready to Move FLOOR Ground out of 2 TRANSACTION New Property FURNISHING Semi-Furnished BATHROOM 1
3,cardid74193103,2 BHK Apartment for Sale in Chettipunyam Chennai,"Strategically located - Right opposite to Mahindra World City, which is home to 67 companies & one lakh employeesTuscany apartments offer you with the best of possibilities - Urban Connectivity & Suburban Tranquility.",₹ 29.2 Lac,"₹4,747 per sqft",SUPER AREA 616 sqft UNDER CONSTRUCTION Poss. by Apr '25 TRANSACTION New Property FURNISHING Unfurnished CAR PARKING 1 Covered BATHROOM 1
4,cardid73511737,2 BHK Apartment for Sale in Kovur Chennai,"Jain Advaya is a project by Jain Housing & Constructions Ltd in Chennai. It is a Under Construction project. Jain Advaya offers some of the most conveniently designed Apartment. Located in Kovur, it is a residential project. It has 122 units. There is 1 building in this project. Jain Advaya offers some of the most exclusive 2 BHK, 3 BHK. As per the area plan, units are in the size range of 733.0 - 1274.0 sq.ft.. Launched in December 2023, Jain Advaya is slated for possession in Dec, 2025. The address of Jain Advaya is Kovur.",₹ 47.7 Lac,"₹6,504 per sqft",SUPER AREA 733 sqft UNDER CONSTRUCTION Poss. by Dec '24 TRANSACTION New Property FURNISHING Unfurnished BATHROOM 2


In [5]:
df.shape

(1080, 6)

In [6]:
# extracting data from title column

# listing_id
df['listing_id'] = df['Listing ID'] = df['Listing ID'].apply(lambda x: x.strip('cardid'))

# neighbourhood
df['neighbourhood'] = df['Title'].str.split(' ').str[-2]

# city
df['city'] = df['Title'].str.split(' ').str[-1]

# no of rooms
df['no_of_rooms'] = df['Title'].str.extract('([0-9]+ BHK)')

In [7]:
# getting property type from title 

def get_property_type(title):
    if 'Apartment' in title:
        return 'Apartment'
    elif 'Builder Floor' in title:
        return 'Builder Floor'
    elif 'Villa' in title:
        return 'Villa'
    elif 'Plot/Land' in title:
        return 'Plot/Land'
    elif 'House' in title:
        return 'House'
    elif 'Penthouse' in title:
        return 'Penthouse'
    else:
        return None


# property type 
df['property_type'] = df['Title'].apply(get_property_type)

In [8]:
# extracting data from summary column

# status
df['status'] = df['Summary'].apply(lambda x: 'Ready to Move' if 'Ready to Move' in x else 'UNDER CONSTRUCTION' if 'UNDER CONSTRUCTION' in x else None)

# furnishing
df['furnish'] = df['Summary'].str.split('FURNISHING ').str[1].str.split(' ').str[0]

# total sqft
df['total_sqft'] = df['Summary'].str.split(' sqft').str[0].str.split(' ').str[-1].replace('Pool', ).astype(int)

# sale type 
df['sale_type'] = df['Summary'].str.extract('TRANSACTION (.*?) FURNISHING')

# price per sqft
df['price_per_sqft'] = df['Sq Ft Price'].str.replace('₹', '').str.replace(',', '').str.replace('Null', '0').str.split(' ').str[0].astype(int)

In [9]:
# price in thousands (k)
def convert_to_thousands(x):
    if 'Lac' in x:
        x = x.replace('₹', '').replace('Lac', '')
        x = float(x) * 100000
    elif 'Cr' in x:
        x = x.replace('₹', '').replace('Cr', '')
        x = float(x) * 10000000
    return x

df["price (in 'k')"] = df['Price'].apply(convert_to_thousands).div(1000).astype(int)

In [10]:
df = df.drop(['Listing ID', 'Title', 'Sq Ft Price','Summary', 'Description'], axis=1)

In [11]:
# Rename column 'OLD_NAME' to 'new_name' for better use
df = df.rename(columns={'Price': 'price'})

In [12]:
# column reorder
df = df[['listing_id', 'city', 'neighbourhood', 'property_type', 'total_sqft','no_of_rooms', 'furnish', 'sale_type', 'status', 'price', "price (in 'k')", 'price_per_sqft']]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   listing_id      1080 non-null   object
 1   city            1080 non-null   object
 2   neighbourhood   1080 non-null   object
 3   property_type   1080 non-null   object
 4   total_sqft      1080 non-null   int32 
 5   no_of_rooms     781 non-null    object
 6   furnish         183 non-null    object
 7   sale_type       183 non-null    object
 8   status          771 non-null    object
 9   price           1080 non-null   object
 10  price (in 'k')  1080 non-null   int32 
 11  price_per_sqft  1080 non-null   int32 
dtypes: int32(3), object(9)
memory usage: 88.7+ KB


In [14]:
df.isnull().sum()

listing_id          0
city                0
neighbourhood       0
property_type       0
total_sqft          0
no_of_rooms       299
furnish           897
sale_type         897
status            309
price               0
price (in 'k')      0
price_per_sqft      0
dtype: int64

In [15]:
# if property is under construction they may not decide whether they are going to furnish it or not it changed to under construction
df.loc[df['status'] == 'UNDER CONSTRUCTION', 'furnish'] = 'under construction'
df.loc[df['property_type'] == 'Plot/Land', 'furnish'] = 'n/a'

In [16]:
# property under constructions are comes under new property  
df.loc[df['status'] == 'UNDER CONSTRUCTION', 'sale_type'] = 'New Property'
df.loc[df['property_type'] == 'Plot/Land', 'sale_type'] = 'n/a'

In [17]:
# set status to n/a for no_of_rooms 
df.loc[df['property_type'] == 'Plot/Land', 'no_of_rooms'] = 'n/a'

In [18]:
# set status to 'n/a' for Plot/Land and fill NaNs with most common status
df.loc[df['property_type'] == 'Plot/Land', 'status'] = 'n/a'
df['status'] = df['status'].fillna(df['status'].mode().max())

In [19]:
# drop duplicates 
df = df.drop_duplicates()

In [20]:
df.isnull().sum()

listing_id          0
city                0
neighbourhood       0
property_type       0
total_sqft          0
no_of_rooms         1
furnish           510
sale_type         510
status              0
price               0
price (in 'k')      0
price_per_sqft      0
dtype: int64

In [21]:
# drop null values 
df.dropna(inplace=True)

In [22]:
df.isnull().sum()

listing_id        0
city              0
neighbourhood     0
property_type     0
total_sqft        0
no_of_rooms       0
furnish           0
sale_type         0
status            0
price             0
price (in 'k')    0
price_per_sqft    0
dtype: int64

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540 entries, 0 to 1078
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   listing_id      540 non-null    object
 1   city            540 non-null    object
 2   neighbourhood   540 non-null    object
 3   property_type   540 non-null    object
 4   total_sqft      540 non-null    int32 
 5   no_of_rooms     540 non-null    object
 6   furnish         540 non-null    object
 7   sale_type       540 non-null    object
 8   status          540 non-null    object
 9   price           540 non-null    object
 10  price (in 'k')  540 non-null    int32 
 11  price_per_sqft  540 non-null    int32 
dtypes: int32(3), object(9)
memory usage: 48.5+ KB


In [24]:
df.shape

(540, 12)

In [25]:
df.head(5)

,listing_id,city,neighbourhood,property_type,total_sqft,no_of_rooms,furnish,sale_type,status,price,price (in 'k'),price_per_sqft
0,71632357,Chennai,Kelambakkam,Apartment,997,2 BHK,Unfurnished,New Property,Ready to Move,₹ 39.9 Lac,3990,4000
1,73795355,Chennai,Mudichur,Builder Floor,873,2 BHK,under construction,New Property,UNDER CONSTRUCTION,₹ 50.6 Lac,5060,5800
2,72552613,Chennai,Kundrathur,Builder Floor,517,1 BHK,Semi-Furnished,New Property,Ready to Move,₹ 27.4 Lac,2740,5300
3,74193103,Chennai,Chettipunyam,Apartment,616,2 BHK,under construction,New Property,UNDER CONSTRUCTION,₹ 29.2 Lac,2920,4747
4,73511737,Chennai,Kovur,Apartment,733,2 BHK,under construction,New Property,UNDER CONSTRUCTION,₹ 47.7 Lac,4770,6504


In [26]:
df.to_csv('cleaned_data.csv', index=False, header=True)